<div class="div_image pull-right">
    <div class = "image image_topic pull-right">
        <img src = "https://i.imgur.com/EGtMXKh.jpg?1">
    </div>
</div>

# <b>Capstone Project: Predicting Dota 2 Match Wins using Machine Learning and Recommender System (Greedy Search Algorithm)</b>


# Import Libraries

In [29]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tqdm

import json

import warnings
warnings.filterwarnings('ignore')


In [30]:
# Import heroes data from json file

with open('heroes.json', 'r') as fp:
    heroes = json.load(fp)
    hero_ids = [hero['id'] for hero in heroes]

def get_hero_human_readable(hero_id):
    for hero in heroes:
        if hero['id'] == hero_id:
            return hero['localized_name']
    return 'Unknown hero: %d' % hero_id


In [31]:
#Store Variables that may change
FINAL_HERO_ID = hero_ids[-1]
NUM_FEATURES = FINAL_HERO_ID * 2
ENV_PATH = 'model.h5'
radiant_team = [8,13, 58, 19,]
dire_team = [70, 12, 55, 105, 51]

In [32]:
print(missing_ids)

[0, 24, 115, 116, 117, 118, 122, 124, 125, 127, 130, 131, 132, 133, 134, 161, 252, 253, 254, 255, 259, 261, 262, 264, 267, 268, 269, 270, 271]


In [33]:
class NNPredictor:
    '''Load Model from the environment path for us to train'''
    def __init__(self, env_path = ENV_PATH):
        self.model = tf.keras.models.load_model(env_path)

    def transform(self, my_team, their_team):
        '''Transform our inputs into the tensorflow input array of shape (1, 246). Slice list of IDs from heroes list '''
        X = np.zeros((1, (NUM_FEATURES+1)))
        '''Slice hero_id chosen onto the array, add number of heroes to hero_id for dire team'''
        for hero_id in my_team:
            X[0][(hero_id)] = 1
        for hero_id in their_team:
            X[0][(hero_id + FINAL_HERO_ID)] = 1
        
        missing_ids = []
        for num in range(0, FINAL_HERO_ID):
            if num not in hero_ids:
                 missing_ids.append(num)
        
        dire_ids = []
        for id in missing_ids:
            if id > 0:
                dire_id = id + FINAL_HERO_ID
                dire_ids.append(dire_id)

        missing_ids = missing_ids + dire_ids

        X = np.delete(X, missing_ids, axis=1)
        return X

    def recommend(self, my_team, their_team, hero_candidates):
        team_possibilities = [(candidate, my_team + [candidate]) for candidate in hero_candidates]

        prob_candidate_pairs = []
        for candidate, team in team_possibilities:
            query = self.transform(team, their_team)
            prob = self.score(query)
            prob_candidate_pairs.append((prob, candidate))
        prob_candidate_pairs = sorted(prob_candidate_pairs, reverse = True)[0:5]
        return prob_candidate_pairs

    def score(self,query):
        rad_prob_1 = self.model.predict(query, verbose = False)[0][0]
        return rad_prob_1

    def predict(self, dream_team, their_team):
        '''Returns the probability of the dream_team winning against their_team.'''
        dream_team_query = self.transform(dream_team, their_team)
        return self.score(dream_team_query)


In [34]:
def main():
    my_team = radiant_team
    their_team = dire_team
    print (f'My Team: {[get_hero_human_readable(hero_id) for hero_id in my_team]}')
    print (f'Their Team:{[get_hero_human_readable(hero_id) for hero_id in their_team]}')
    print ('Recommend:')

    engine = Engine(NNPredictor())
    recommendations = engine.recommend(my_team, their_team)
    print(f'{[(prob, get_hero_human_readable(hero))for prob, hero in recommendations]}')

In [35]:
class Engine:

    def __init__(self, algorithm):
        self.algorithm = algorithm

    def get_candidates(self, my_team, their_team):
        '''Return a list of hero_IDs to consider for recommending'''
        ids = [i for i in hero_ids if i not in my_team and i not in their_team and i not in [
            24, 115, 116, 117, 118, 122, 124, 125, 127, 130, 131, 132, 133, 134]]
        return ids

    def recommend(self, my_team, their_team, human_readable = False):
        '''Return a list of (hero, probability of winning with hero_added) recommend to complete my_team'''

        assert len(my_team) <= 5
        assert len(their_team) <= 5

        hero_candidates = self.get_candidates(my_team, their_team)
        return self.algorithm.recommend(my_team, their_team, hero_candidates)

    def predict(self, dream_team, their_team):
        '''Return the probability of the dream_team winning against their team'''
        return self.algorithm.predict(dream_team, their_team)



In [36]:
if __name__ == '__main__':
    main()

My Team: ['Juggernaut', 'Puck', 'Enchantress', 'Tiny']
Their Team:['Ursa', 'Phantom Lancer', 'Dark Seer', 'Techies', 'Clockwerk']
Recommend:
[(0.9309222, 'Arc Warden'), (0.8738238, 'Shadow Demon'), (0.86496115, 'Naga Siren'), (0.86416465, 'Faceless Void'), (0.85787666, 'Sniper')]


In [37]:
test_model  = tf.keras.models.load_model('model.h5')

In [38]:
radiant_team = [8,13, 58, 19, 14]
dire_team = [70, 12, 55, 105, 51]

In [39]:
test = np.zeros((1, (NUM_FEATURES+1)))
for hero_id in radiant_team:
    test[0][(hero_id)] = 1
for hero_id in dire_team:
    test[0][(hero_id + FINAL_HERO_ID)] = 1
test = np.delete(test,[0, 24, 115, 116, 117, 118, 122, 124, 125, 127, 130, 131, 132, 133, 134, 161, 252, 253, 254, 255, 259, 261, 262, 264, 267, 268, 269, 270, 271],axis=1)


In [40]:
len(test[0])

246

In [41]:
test_pred = test_model.predict(test)

print(test_pred)

1/1 [==============================] - 0s 120ms/step
[[0.5129051]]
